In [1]:
import time
import requests
from bs4 import BeautifulSoup

In [97]:
def get_news():
    news_list = []
    start_page = 1
    
    while start_page < 10:
        url = f'https://tengrinews.kz/news/page/{start_page}/'
        response = requests.get(url)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            for content_main_item in soup.find_all('div', class_='content_main_item'):
    
                text_date = list(
                    content_main_item.find('div', class_='content_main_item_meta')
                )[1].text.strip()
                
                if text_date not in ('Вчера', 'Сегодня'):
                    break
                
                if text_date == 'Вчера':
                    href = content_main_item.find('a').get('href')
                    category = href.split('/')[1]
                    item_id = href[:-1].split('-')[-1]
                    news = {
                        'page_url': f'https://tengrinews.kz'+href,
                        'category': category,
                        'page_id': item_id
                    }
                    news_list.append(news)
                
            if text_date not in ('Вчера', 'Сегодня'):
                break
                    
        else:
            raise Exception(f'Failed to retrieve page {page_number}')
    
        start_page += 1
        time.sleep(0.5)

    return news_list

In [98]:
%%time

get_news()

CPU times: total: 2.36 s
Wall time: 5.54 s


[{'page_url': 'https://tengrinews.kz/kazakhstan_news/aktobe-reklamnyiy-schit-upal-ostanovku-postradali-neskolko-532710/',
  'category': 'kazakhstan_news',
  'page_id': '532710'},
 {'page_url': 'https://tengrinews.kz/kazakhstan_news/opasnyie-igryi-devochka-okazalas-reanimatsii-upav-tretego-532705/',
  'category': 'kazakhstan_news',
  'page_id': '532705'},
 {'page_url': 'https://tengrinews.kz/kazakhstan_news/oplyivina-soshla-v-almatyi-532706/',
  'category': 'kazakhstan_news',
  'page_id': '532706'},
 {'page_url': 'https://tengrinews.kz/kazakhstan_news/tri-oblasti-i-semey-ormanyi-ne-gotovyi-k-lesnyim-pojaram-532704/',
  'category': 'kazakhstan_news',
  'page_id': '532704'},
 {'page_url': 'https://tengrinews.kz/kazakhstan_news/astane-zaderjali-inostrantsa-razyiskivaemogo-torgovlyu-532703/',
  'category': 'kazakhstan_news',
  'page_id': '532703'},
 {'page_url': 'https://tengrinews.kz/kazakhstan_news/chto-byilo-na-sude-po-delu-bishimbaeva-17-aprelya-532660/',
  'category': 'kazakhstan_news'

In [138]:
import requests
from bs4 import BeautifulSoup

def get_news(url) -> dict:
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to retrieve the page")
        return {}
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    news_title = soup.find('h1', class_='head-single').get_text(strip=True)
    news_content_soup = soup.find('div', class_='content_main_text')
    
    p_tags = news_content_soup.find_all('p')
    last_p = p_tags[-1]
    last_p.decompose()

    hashtags_soup = news_content_soup.find('div', class_='content_main_text_tags')
    hashtags = ', '.join([i.text.lower() for i in hashtags_soup.find_all('a')])
    hashtags_soup.decompose()
    
    news_content = news_content_soup.get_text(strip=True)
    news_date = soup.find('div', class_='date-time').get_text(strip=True)

    return {
        'title': news_title,
        'content': news_content,
        'hashtags': hashtags,
        'date': news_date
    }

def get_views(news_id) -> int:
    response = requests.get(f'https://counter.tengrinews.kz/inc/tengrinews_ru/news/{news_id}')
    if response.status_code == 200:
        return a.json()['result']

    return None

In [139]:
# Example usage
url = 'https://tengrinews.kz/kazakhstan_news/aktobe-reklamnyiy-schit-upal-ostanovku-postradali-neskolko-532710/'
news_data = scrape_news(url)
print(news_data)

{'title': 'В Актобе рекламный щит упал на остановку. Пострадали несколько человек', 'content': 'В Актобе вечером 17 апреля рекламный щит упал на автобусную остановку, на которой находились люди. После происшествия госпитализированы три женщины, передаетTengrinews.kzсо ссылкой на "Диапазон".Инцидент произошел на проспекте А. Молдагуловой в районе ЖК "Золотой квартал". При установке рекламного щита на стеклянную крышу остановки рухнула часть конструкции. Под осколки разрушившейся крыши попали три женщины, которые ждали автобус."Пострадавших госпитализировали. Началось досудебное расследование по факту выполнения работ либо оказания услуг, не отвечающих требованиям безопасности", - рассказали в департаменте полиции Актюбинской области.', 'hashtags': 'актобе, происшествие, травмы', 'date': 'Вчера 23:54'}


In [140]:
import datetime

def parse_russian_date(date_str):
    # Russian month names mapping to their numeric values
    months = {
        'января': 1, 'февраля': 2, 'марта': 3,
        'апреля': 4, 'мая': 5, 'июня': 6,
        'июля': 7, 'августа': 8, 'сентября': 9,
        'октября': 10, 'ноября': 11, 'декабря': 12
    }
    
    # Current date
    today = datetime.datetime.now()
    
    # Handling relative dates
    if 'Сегодня' in date_str:
        date = today
        time_part = date_str.split()[1]
    elif 'Вчера' in date_str:
        date = today - datetime.timedelta(days=1)
        time_part = date_str.split()[1]
    else:
        # Splitting the date string by space
        day, month, year, time_part = date_str.split()
        day = int(day)
        year = int(year)
        month = months[month]
        date = datetime.datetime(year, month, day)
    
    # Parsing the time
    hour, minute = map(int, time_part.split(':'))
    
    # Combining date and time
    return datetime.datetime(date.year, date.month, date.day, hour, minute)

# Test the function
dates = [
    "16 апреля 2024 12:28",
    "03 мая 2024 12:28",
    "Сегодня 17:02",
    "Вчера 14:44"
]

parsed_dates = [parse_russian_date(date) for date in dates]
for date in parsed_dates:
    print(date)


2024-04-16 12:28:00
2024-05-03 12:28:00
2024-04-18 17:02:00
2024-04-17 14:44:00


In [144]:
{'1': 1} | {'2': 2}

{'1': 1, '2': 2}

In [147]:
all([True, True, True])

True